# Basic Pushbroom Imager on a Satellite (Pléiades example)

Computing the basic parameters of the satellite pushbroom imager on the French high-res satellite Pléiades.

Sources for the Pléiades data:

- [ESA Pléiades page](https://earth.esa.int/eogateway/missions/pleiades#instruments-section)
- [eoPortal Pléiades page](https://www.eoportal.org/satellite-missions/pleiades#hiri-high-resolution-imager)

## Loading the Imager Parameters

An imager is made up of three parts: Optics, Detector and (optionally) Read-out/Write Electronics. We load the configuration files for each part and initialise the `Imager` object.

In [22]:
from pathlib import Path

from opticks.imager_model.detector import Channel
from opticks.imager_model.imager import Imager

print(f"current working directory: {Path.cwd()}")

file_directory = Path("docs", "examples", "sample_sat_pushbroom")
optics_file_path = file_directory.joinpath("optics.yaml")
detector_file_path = file_directory.joinpath("pan_detector.yaml")
rw_electronics_file_path = file_directory.joinpath("rw_electronics.yaml")

# check whether input files exist
print(
    f"optics file:  [{optics_file_path}] (file exists:  {optics_file_path.is_file()})"
)
print(
    f"detector file exists:  [{detector_file_path}] (file exists:  {detector_file_path.is_file()})"
)
print(
    f"RW electronics file exists:  [{rw_electronics_file_path}] (file exists:  {rw_electronics_file_path.is_file()})"
)

# Init imager object
imager = Imager.from_yaml_file(
    optics_file_path, detector_file_path, rw_electronics_file_path
)

# shorthands
optics = imager.optics
detector = imager.detector
rw_electronics = imager.rw_electronics

# select the PAN channel
band_id = "pan"
channel: Channel = detector.params.channels.all[band_id]

# binning status
binning_on = False if channel.binning == 1 else True

current working directory: /home/egemen/Projects/opticks
optics file:  [docs/examples/sample_sat_pushbroom/optics.yaml] (file exists:  True)
detector file exists:  [docs/examples/sample_sat_pushbroom/pan_detector.yaml] (file exists:  True)
RW electronics file exists:  [docs/examples/sample_sat_pushbroom/rw_electronics.yaml] (file exists:  True)


## Setting the Scene

Reference parameters for the camera position and motion with respect to the target are given below:

In [23]:
import numpy as np

from opticks import u

# constants
# ---------
r_earth = 6378.137 * u.km
mu = 398600.5 * u.km**3 / u.s**2

# sat positional params
# ---------------------
sat_altitude = 694.0 * u.km

n = np.sqrt(mu / (r_earth + sat_altitude) ** 3)
ground_vel = (n * r_earth).to("m/s")

# converted to generic params
distance = sat_altitude
target_rel_velocity = ground_vel

print(f"target distance : {distance:~P} ")
print(f"ground velocity : {target_rel_velocity:~P.6}")

target distance : 694.0 km 
ground velocity : 6770.75 m/s


Certain parameters vary with the light characteristics and therefore wavelength dependent. Pléiades PAN works in the range 480 to 820 nm. We will use the centre frequency as the reference wavelength.

Note the widely used bands:

- blue: 450-485 nm
- green: 500-565 nm
- red: 625-750 nm
- nir: 750-1400 nm

In [24]:
ref_wavelength = channel.centre_freq

print(f"reference wavelength : {ref_wavelength:~}")

reference wavelength : 650.0 nm


## Extracting the Imager Parameters

### Optical Parameters

Basic and derived optical parameters are given below for the optics:


In [25]:
print("Basic Optical Params:")

print(f"optics id: {optics.params.name}")
print(f"focal length : {optics.params.focal_length:~}")
print(f"aperture diameter : {optics.params.aperture_diameter:~}")
print(f"image diameter on focal plane : {optics.params.image_diam_on_focal_plane:~}")

print()
print("Derived Optical Params:")

print(f"f-number : {optics.f_number:.4}")
print(f"full optical fov : {optics.full_optical_fov:~P.4}")
print(f"aperture area : {optics.aperture_area.to("cm**2"):~P.6}")
print(f"spatial cut-off freq  : {optics.spatial_cutoff_freq(ref_wavelength):~P.5} (at {ref_wavelength:~P})")

Basic Optical Params:
optics id: Pléiades TMA Optics
focal length : 12905 mm
aperture diameter : 650 mm
image diameter on focal plane : 400 mm

Derived Optical Params:
f-number : 19.85
full optical fov : 1.776 deg
aperture area : 3318.31 cm²
spatial cut-off freq  : 77.489 cy/mm (at 650.0 nm)


### Detector Parameters

Basic and derived detector parameters are given below for the optics:

In [26]:
print("Basic Detector Params:")

print(f"detector id: {detector.params.name}")
print(f"detector type : {detector.params.detector_type}")
print(f"horizontal x vertical pixels (detector) : {detector.params.horizontal_pixels} x {detector.params.vertical_pixels}")
print(f"horizontal x vertical pixels (channel) : {channel.horizontal_pixels} x {channel.vertical_pixels}")
print(f"binning : {channel.binning if binning_on else 'None'}")
print(f"pixel pitch : {channel.pixel_pitch(False):~P} {f"({channel.pixel_pitch(True):~P} binned)" if binning_on else ''}")
print(f"TDI stages : {'None' if channel.tdi_stages == 1 else channel.tdi_stages}")

print()
print("Derived Detector Params:")

print(f"Nyquist freq : {channel.nyquist_freq(False):~P.4} {f"({channel.nyquist_freq(True):~P.4} binned)" if binning_on else ''}")
print(f"number of pixels (full frame) : {detector.pixel_count:~P.4}")
print(f"number of pixels (full frame, channel) : {channel.pixel_count_frame(False):~P.4} {f"({channel.pixel_count_frame(True):~P.4} binned)" if binning_on else ''}")

Basic Detector Params:
detector id: Pléiades PAN Detector
detector type : pushbroom
horizontal x vertical pixels (detector) : 30000 x 20
horizontal x vertical pixels (channel) : 30000 x 20
binning : None
pixel pitch : 13 µm 
TDI stages : 13

Derived Detector Params:
Nyquist freq : 38.46 cy/mm 
number of pixels (full frame) : 0.6 Mpixel
number of pixels (full frame, channel) : 0.6 Mpixel 


### Imager Geometry Parameters

The derived parameters are given below for the imager:

In [27]:
print(f"ifov : {imager.ifov(band_id, False):~P.4} {f"({imager.ifov(band_id, True):~P.4} binned)" if binning_on else ''}")
print(f"pixel solid angle : {imager.pix_solid_angle(band_id, False):~P.4} {f"({imager.pix_solid_angle(band_id, True):~P.4} binned)" if binning_on else ''}")
print(f"horizontal full fov: {imager.horizontal_fov(band_id):~P.4}")
print(f"vertical full fov: {imager.vertical_fov(band_id):~P.4}")

ifov : 0.05771 mdeg 
pixel solid angle : 1.015×10⁻¹² sr 
horizontal full fov: 1.731 deg
vertical full fov: 0.001154 deg


### Geometric Projection Parameters

In [28]:
# swath assuming flat plate and constant Instantaneous FoV
swath = imager.projected_horiz_img_extent(distance, band_id)
 
print(f"image swath : {swath:~.4P} (disregarding Earth curvature)")

# Ground sample distance at Nadir
if binning_on:
    ssd_nadir_binned = imager.spatial_sample_distance(distance, band_id, True, "centre")
    print(f"ssd at nadir (binned): {ssd_nadir_binned.horiz:~.4P} , {ssd_nadir_binned.vert:~.4P} (horizontal, vertical at target distance)")

ssd_nadir = imager.spatial_sample_distance(distance, band_id, False, "centre")
print(f"ssd at nadir (unbinned): {ssd_nadir.horiz:~.4P} , {ssd_nadir.vert:~.4P} (horizontal, vertical at target distance)")

image swath : 20.97 km (disregarding Earth curvature)
ssd at nadir (unbinned): 0.6991 m , 0.6991 m (horizontal, vertical at target distance)


### Timings

In [29]:
timings = detector.params.timings


print(f"line duration : {channel.frame_duration:~P.4} ({channel.frame_rate:~P.6}) (with binning where applicable)")
print(f"max integration duration : {timings.max_integration_duration:~P.4} (no binning)")
print(f"actual integration duration : {timings.integration_duration:~P.4} (no binning)")
print(f"total TDI column duration : {channel.total_tdi_col_duration:~P.4} ({channel.tdi_stages}x stages) (with binning where applicable)")

line duration : 0.1019 ms (9811.6 Hz) (with binning where applicable)
max integration duration : 0.1019 ms (no binning)
actual integration duration : 0.03 ms (no binning)
total TDI column duration : 1.325 ms (13x stages) (with binning where applicable)


### Read-out/Write Electronics

In [30]:
print(
    f"pixel read rate (without TDI) : {detector.pix_read_rate(band_id, False, False):~P.6} {f'({detector.pix_read_rate(band_id, True, False):~P.6} binned)' if binning_on else ''} ({channel.name} channel)"
)
print(
    f"pixel read rate (with TDI) : {detector.pix_read_rate(band_id, False, True):~P.6} {f'({detector.pix_read_rate(band_id, True, True):~P.6} binned)' if binning_on else ''} ({channel.name} channel)"
)

print(
    f"data write rate (uncompressed, incl. overheads) : {imager.data_write_rate(band_id, False, False):~P.6} {f'({imager.data_write_rate(band_id, True, False):~P.6} binned)' if binning_on else ''} ({channel.name} channel)"
)
print(
    f"data write rate (compressed, incl. overheads) : {imager.data_write_rate(band_id, False, True):~P.6} {f'({imager.data_write_rate(band_id, True, True):~P.6} binned)' if binning_on else ''} ({channel.name} channel)"
)

pixel read rate (without TDI) : 294.348 Mpixel/s  (PAN channel)
pixel read rate (with TDI) : 3826.52 Mpixel/s  (PAN channel)
data write rate (uncompressed, incl. overheads) : 3638.14 Mbit/s  (PAN channel)
data write rate (compressed, incl. overheads) : 909.535 Mbit/s  (PAN channel)


## References

1. [Design of the high resolution optical instrument for the Pleiades HR Earth observation satellites](https://www.semanticscholar.org/paper/Design-of-the-high-resolution-optical-instrument-HR-Lamard-Gaudin-Delrieu/d8cdc4643577da00edc210a49eeb66fa48a71374)